Autor: Werikson Alves

Codigo para calulo para fluxo de potencias pelo metodo de Newton-Raphson

In [1]:
import numpy as np
from sympy import * #var, Lambda, exp, log, sin, cos, tan, sqrt, ln

Parametros relacionados com a impedancia

In [2]:
#Dados iniciais
Barras = 3
Impedancias = 2

#                [      Tipo, V, teta,   P,   Q]
Matriz_Barras = [[1, 'VTeta', 1,   0,  '?',   '?'],
                 [2, 'PV',    1, '?',  0.2,   '?'],
                 [3, 'PQ',  '?', '?', -0.15, 0.05]]

#                [     k, m, Z_km]
Matriz_Impedancias = [[1, 2, complex(0.012, 0.3)],
                      [2, 3, complex(0.011, 0.8)]]

y11 =  (Matriz_Impedancias[0][2])**(-1)
y12 = -(Matriz_Impedancias[0][2])**(-1)
y13 =  0
y21 =  y12
y22 =  (Matriz_Impedancias[0][2])**(-1) + (Matriz_Impedancias[1][2])**(-1)
y23 = -(Matriz_Impedancias[1][2])**(-1)
y31 =  y13
y32 =  y23
y33 =  (Matriz_Impedancias[1][2])**(-1)

Y = np.array([[y11, y12,y13], [y21, y22, y23], [y31, y32, y33]])

G = Y.real
B = Y.imag

print('\nY = ', Y)
print('\nG = ', G)
print('\nB = ', B)


Y =  [[ 0.13312034-3.32800852j -0.13312034+3.32800852j  0.        +0.j        ]
 [-0.13312034+3.32800852j  0.15030459-4.57777224j -0.01718425+1.24976372j]
 [ 0.        +0.j         -0.01718425+1.24976372j  0.01718425-1.24976372j]]

G =  [[ 0.13312034 -0.13312034  0.        ]
 [-0.13312034  0.15030459 -0.01718425]
 [ 0.         -0.01718425  0.01718425]]

B =  [[-3.32800852  3.32800852  0.        ]
 [ 3.32800852 -4.57777224  1.24976372]
 [ 0.          1.24976372 -1.24976372]]


Parametros relacionados com as potencias

In [40]:
#Variáveis de entrada
teta2,teta3,V3 = var('teta2 teta3 V3')
teta1, V1, V2 = 0, 1, 1

# Potências esperadas nas barras 
P_esp_2 = Matriz_Barras[1][4]
P_esp_3 = Matriz_Barras[2][4]
Q_esp_3 = Matriz_Barras[2][5]

# Vetor de potencias
P2 = Lambda((teta2,teta3,V3), V2*( V1*(G[1,0]*cos(teta2-teta1) + B[1,0]*sin(teta2-teta1)) +
                                   V3*(G[1,2]*cos(teta2-teta3) + B[1,2]*sin(teta2-teta3)) + 
                                   V2*(G[1,1]*cos(teta2-teta2) + B[1,1]*sin(teta2-teta2)) ))

P3 = Lambda((teta2,teta3,V3), V3*( V3*(G[2,2]*cos(teta3-teta3) + B[2,2]*sin(teta3-teta3)) + 
                                   V2*(G[2,1]*cos(teta3-teta2) + B[2,1]*sin(teta3-teta2)) ))

Q3 = Lambda((teta2,teta3,V3), V3*( V3*(G[2,2]*sin(teta3-teta3) - B[2,2]*cos(teta3-teta3)) + 
                                   V2*(G[2,1]*sin(teta3-teta2) - B[2,1]*cos(teta3-teta2)) ))

Delta_P_Q = np.array([[P_esp_2 - P2(teta2,teta3,V3)], 
                      [P_esp_3 - P3(teta2,teta3,V3)], 
                      [Q_esp_3 - Q3(teta2,teta3,V3)]])

for i in range(len(Delta_P_Q)):
    print('\nDelta_P_Q[',i+1,'] = ', Delta_P_Q[i])


Delta_P_Q[ 1 ] =  [-V3*(1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3)) - 3.32800851970181*sin(teta2) + 0.133120340788072*cos(teta2) + 0.0496954081094017]

Delta_P_Q[ 2 ] =  [-V3*(0.0171842511025259*V3 - 1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3)) - 0.15]

Delta_P_Q[ 3 ] =  [-V3*(1.24976371654734*V3 + 0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3)) + 0.05]


Parametros relacionados com a Jacobiana

In [9]:
# Matriz Jacobiana
j11 = Lambda((teta2,teta3,V3), diff(P_esp_2 - P2(teta2,teta3,V3),teta2))
j12 = Lambda((teta2,teta3,V3), diff(P_esp_2 - P2(teta2,teta3,V3),teta3))
j13 = Lambda((teta2,teta3,V3), diff(P_esp_2 - P2(teta2,teta3,V3),V3))

j21 = Lambda((teta2,teta3,V3), diff(P_esp_3 - P3(teta2,teta3,V3),teta2))
j22 = Lambda((teta2,teta3,V3), diff(P_esp_3 - P3(teta2,teta3,V3),teta3))
j23 = Lambda((teta2,teta3,V3), diff(P_esp_3 - P3(teta2,teta3,V3),V3))

j31 = Lambda((teta2,teta3,V3), diff(Q_esp_3 - Q3(teta2,teta3,V3),teta2))
j32 = Lambda((teta2,teta3,V3), diff(Q_esp_3 - Q3(teta2,teta3,V3),teta3))
j33 = Lambda((teta2,teta3,V3), diff(Q_esp_3 - Q3(teta2,teta3,V3),V3))

J = np.array([[j11(teta2,teta3,V3), j12(teta2,teta3,V3), j13(teta2,teta3,V3)],
              [j21(teta2,teta3,V3), j22(teta2,teta3,V3), j23(teta2,teta3,V3)],
              [j31(teta2,teta3,V3), j32(teta2,teta3,V3), j33(teta2,teta3,V3)]])

#J_inv = np.linalg.inv(J)

for i in range(len(J)):
    for j in range(len(J)):
        print('J[',i+1,'][',j+1,'] = ', J[i][j])
    print('\n')


J[ 1 ][ 1 ] =  -V3*(0.0171842511025259*sin(teta2 - teta3) + 1.24976371654734*cos(teta2 - teta3)) - 0.133120340788072*sin(teta2) - 3.32800851970181*cos(teta2)
J[ 1 ][ 2 ] =  -V3*(-0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3))
J[ 1 ][ 3 ] =  -1.24976371654734*sin(teta2 - teta3) + 0.0171842511025259*cos(teta2 - teta3)


J[ 2 ][ 1 ] =  -V3*(0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3))
J[ 2 ][ 2 ] =  -V3*(-0.0171842511025259*sin(teta2 - teta3) + 1.24976371654734*cos(teta2 - teta3))
J[ 2 ][ 3 ] =  -0.0343685022050519*V3 + 1.24976371654734*sin(teta2 - teta3) + 0.0171842511025259*cos(teta2 - teta3)


J[ 3 ][ 1 ] =  -V3*(1.24976371654734*sin(teta2 - teta3) + 0.0171842511025259*cos(teta2 - teta3))
J[ 3 ][ 2 ] =  -V3*(-1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3))
J[ 3 ][ 3 ] =  -2.49952743309468*V3 - 0.0171842511025259*sin(teta2 - teta3) + 1.24976371654734*cos(teta2 - teta3)




In [7]:
# Vetor inicial
# teta2, teta3, V3 = 0, 0, 1
teta2_inicial, teta3_inicial, V3_inicial = 0, 0, 1
X = np.array([[teta2_inicial], 
              [teta3_inicial],
              [V3_inicial]])

Delta_teta_V = np.array([[teta2_inicial + teta2], 
                         [teta3_inicial + teta3], 
                         [V3_inicial + V3]])

# Erro
Ep = 0.001
Eq = 0.001

# Numeros de iterações
iteracao = 0

In [36]:
def substitution(Funcao, X):
    return Subs(Funcao, (teta2,teta3,V3), X)

'''Delta_P_Q = np.array([[P_esp_2 - P2(teta2,teta3,V3)], 
                      [P_esp_3 - P3(teta2,teta3,V3)], 
                      [Q_esp_3 - Q3(teta2,teta3,V3)]])'''



'Delta_P_Q = np.array([[P_esp_2 - P2(teta2,teta3,V3)], \n                      [P_esp_3 - P3(teta2,teta3,V3)], \n                      [Q_esp_3 - Q3(teta2,teta3,V3)]])'

In [43]:
print('\nIteração: ', iteracao)
print('\nX = ', X)
print('\nDelta_teta_V = ', substitution(Delta_teta_V, X))
print('\nDelta_P_Q = ', substitution(Delta_P_Q, X))
print('\nJ = ', substitution(J, X))



Iteração:  1

X =  [[0]
 [0]
 [1]]

Delta_teta_V =  Subs([[teta2], [teta3], [V3 + 1]], (teta2, teta3, V3), ([0], [0], [1]))

Delta_P_Q =  Subs([[-V3*(1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3)) - 3.32800851970181*sin(teta2) + 0.133120340788072*cos(teta2) + 0.0496954081094017], [-V3*(0.0171842511025259*V3 - 1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3)) - 0.15], [-V3*(1.24976371654734*V3 + 0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3)) + 0.05]], (teta2, teta3, V3), ([0], [0], [1]))

J =  Subs([[-V3*(0.0171842511025259*sin(teta2 - teta3) + 1.24976371654734*cos(teta2 - teta3)) - 0.133120340788072*sin(teta2) - 3.32800851970181*cos(teta2), -V3*(-0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3)), -1.24976371654734*sin(teta2 - teta3) + 0.0171842511025259*cos(teta2 - teta3)], [-V3*(0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3)), -V3*(-0.01718425

In [38]:
#Algoritmo de Newton-Raphson
while(True):
    iteracao += 1
    print('\nIteração: ', iteracao)
    print('\nX = ', X)
    print('\nDelta_teta_V = ', Delta_teta_V)
    print('\nDelta_P_Q = ', Delta_P_Q)
    print('\nJ = ', J)

    # Resposta
    X_resp = X - np.dot(np.linalg.inv(J), Delta_P_Q)
    print('\nX_resp = ', X_resp)

    # Erro
    Erro_teta_V = np.linalg.norm(Delta_teta_V - X_resp[0:3])
    Erro_P_Q = np.linalg.norm(Delta_P_Q)

    print('\nErro_teta_V = ', Erro_teta_V)
    print('\nErro_P_Q = ', Erro_P_Q)

    # Verificação do erro
    if Erro_teta_V < Ep and Erro_P_Q < Eq:
        break
    else:
        X = X_resp


Iteração:  1

X =  [[0]
 [0]
 [1]]

Delta_teta_V =  [[teta2]
 [teta3]
 [V3 + 1]]

Delta_P_Q =  [[-V3*(1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3)) - 3.32800851970181*sin(teta2) + 0.133120340788072*cos(teta2) + 0.0496954081094017]
 [-V3*(0.0171842511025259*V3 - 1.24976371654734*sin(teta2 - teta3) - 0.0171842511025259*cos(teta2 - teta3)) - 0.15]
 [-V3*(1.24976371654734*V3 + 0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3)) + 0.05]]

J =  [[-V3*(0.0171842511025259*sin(teta2 - teta3) + 1.24976371654734*cos(teta2 - teta3)) - 0.133120340788072*sin(teta2) - 3.32800851970181*cos(teta2)
  -V3*(-0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3))
  -1.24976371654734*sin(teta2 - teta3) + 0.0171842511025259*cos(teta2 - teta3)]
 [-V3*(0.0171842511025259*sin(teta2 - teta3) - 1.24976371654734*cos(teta2 - teta3))
  -V3*(-0.0171842511025259*sin(teta2 - teta3) + 1.24976371654734*cos(teta2 - teta3))
  -0.0343685022050519

TypeError: No loop matching the specified signature and casting was found for ufunc inv